In [1]:
from sklearn.metrics import mean_absolute_error as mae

In [163]:
def pickandplace(move):
    #Getting the x-coordinate for initial location
    if move[0]=='E': ##MIDDLE
        src_x = 0
    elif move[0]=='D':
        src_x = -0.028 ##2.8 cm
    elif move[0]=='C':
        src_x = -2*0.028
    elif move[0]=='B':
        src_x = -3*0.028
    elif move[0]=='A':
        src_x = -4*0.028
    elif move[0]=='F':
        src_x = +0.028
    elif move[0]=='G':
        src_x = +2*0.028
    elif move[0]=='H':
        src_x = +3*0.028
    
    #Getting the y coordinate (forward) for initial location
    if move[1]=='1': ##MIDDLE
        src_y = 0
    elif move[1]=='2':
        src_y = 0.028 ##2.8 cm
    elif move[1]=='3':
        src_y = 2*0.028
    elif move[1]=='4':
        src_y = 3*0.028
    elif move[1]=='5':
        src_y = 4*0.028
    elif move[1]=='6':
        src_y = 5*0.028
    elif move[1]=='7':
        src_y = 6*0.028
    elif move[1]=='8':
        src_y = 7*0.028
        
    #Getting the x-coordinate for destination
    if move[2]=='E': ##MIDDLE
        dest_x = 0
    elif move[2]=='D':
        dest_x = -0.028 ##2.8 cm
    elif move[2]=='C':
        dest_x = -2*0.028
    elif move[2]=='B':
        dest_x = -3*0.028
    elif move[2]=='A':
        dest_x = -4*0.028
    elif move[2]=='F':
        dest_x = +0.028
    elif move[2]=='G':
        dest_x = +2*0.028
    elif move[2]=='H':
        dest_x = +3*0.028
    
    #Getting the y coordinate (forward) for destination
    if move[3]=='1': ##MIDDLE
        dest_y = 0
    elif move[3]=='2':
        dest_y = 0.028 ##2.8 cm
    elif move[3]=='3':
        dest_y = 2*0.028
    elif move[3]=='4':
        dest_y = 3*0.028
    elif move[3]=='5':
        dest_y = 4*0.028
    elif move[3]=='6':
        dest_y = 5*0.028
    elif move[3]=='7':
        dest_y = 6*0.028
    elif move[3]=='8':
        dest_y = 7*0.028
        
    
        
        
    return [src_x,src_y], [dest_x,dest_y]

In [191]:
import serial.tools.list_ports

ports=serial.tools.list_ports.comports()
serialInst=serial.Serial()
portsList=[]

for onePort in ports:
    portsList.append(str(onePort))
    print(str(onePort))
    
val=input("Select Port: COM")
for x in range(0,len(portsList)):
    if portsList[x].startswith("COM"+str(val)):
        portVar='COM'+str(val)
        print(portVar)
        
serialInst.baudrate=9600
serialInst.port=portVar
serialInst.open()

COM4 - Standard Serial over Bluetooth link (COM4)
COM3 - Standard Serial over Bluetooth link (COM3)
COM5 - Arduino Mega 2560 (COM5)
Select Port: COM5
COM5


In [169]:
def sendCommand2(a,b,c,d,e):
    command = '{:.3f} {:.3f} {:.3f} {:.3f} {:.3f}\n'.format(a,b,c,d,e)
    serialInst.write(command.encode('ASCII'))
    return command.encode('ASCII')

In [197]:
import pybullet as p
import pybullet_data
import time
from sklearn.metrics import mean_absolute_error as mae
import time

# Set the interval in seconds for sending commands
command_interval = 0.3  # Send commands every 0.5 seconds
last_command_time = time.time()  # Initialize the last command time
last_command = [0,0,0,0,0]
urdf = 'C:/Users/USER/Desktop/Chessbot/five_dof_servo_arm_urdf_compatible_description/five_dof_servo_arm_urdf_compatible_description/urdf/five_dof_servo_arm_urdf_compatible.xacro'

move = "G7C4"
src,dest = pickandplace(move)

print(src)
### Source
# Connect to PyBullet physics simulation with GUI
p.connect(p.GUI)

# Set the path to PyBullet's built-in data
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the robot URDF model (replace 'urdf_file' with the actual URDF file path)
#urdf = "urdf_file.urdf"  # Replace with the actual URDF file path
robot_id = p.loadURDF(urdf, useFixedBase=True)

# Optionally, load a plane to give the scene a ground
plane_id = p.loadURDF("plane.urdf")

# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

# Set initial camera position
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])

# Get the number of joints in the robot
num_joints = p.getNumJoints(robot_id)

move = "G7B4"
src,dest = pickandplace(move)
# Filter out the fixed joints
movable_joints = []
for i in range(num_joints):
    joint_info = p.getJointInfo(robot_id, i)
    #print(joint_info)
    joint_type = joint_info[2]
    if joint_type not in [p.JOINT_FIXED]:  # Only add non-fixed joints
        movable_joints.append(i)

# Create sliders for real-time input of end-effector's x, y, z coordinates (IK control)
x_slider = p.addUserDebugParameter("End Effector X", -1, 1, 0)
y_slider = p.addUserDebugParameter("End Effector Y", -1, 1, 0)
z_slider = p.addUserDebugParameter("End Effector Z", 0, 1, 0.5)

# Create sliders for end-effector orientation (roll, pitch, yaw)
roll_slider = p.addUserDebugParameter("End Effector Roll", -3.14, 3.14, 0)  # Roll in radians
pitch_slider = p.addUserDebugParameter("End Effector Pitch", -3.14, 3.14, 0)  # Pitch in radians
yaw_slider = p.addUserDebugParameter("End Effector Yaw", -3.14, 3.14, 0)  # Yaw in radians

# Define end effector link index (adjust based on your URDF's end effector)
end_effector_link_index = 18  # Modify this based on your actual end-effector link index

# Create a visual sphere to represent the target position
target_visual_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[1, 0, 0, 1])  # Red sphere
target_sphere = p.createMultiBody(baseVisualShapeIndex=target_visual_shape)

# Initialize variables for text IDs
target_text_id = None
end_effector_text_id = None
joint_angle_text_ids = [None] * len(movable_joints)  # For joint angle display text IDs

# Main simulation loop
while True:
    # Read the target position from the sliders
#     target_x = p.readUserDebugParameter(x_slider)
#     target_y = p.readUserDebugParameter(y_slider)
#     target_z = p.readUserDebugParameter(z_slider)

    target_x = src[0]
    target_y = src[1]+0.045+2*0.028
    target_z = 0.1
    target_position = [target_x, target_y, target_z]

    # Read the target orientation from the sliders
#     target_roll = p.readUserDebugParameter(roll_slider)
#     target_pitch = p.readUserDebugParameter(pitch_slider)
#     target_yaw = p.readUserDebugParameter(yaw_slider)
    target_roll = -0.8
    target_pitch = 0
    target_yaw = 0

    # Convert roll, pitch, yaw to quaternion
    target_orientation = p.getQuaternionFromEuler([target_roll, target_pitch, target_yaw])

    # Move the visual sphere to the target position
    p.resetBasePositionAndOrientation(target_sphere, target_position, [0, 0, 0, 1])

    # Use inverse kinematics to calculate joint positions
    ik_joint_positions = p.calculateInverseKinematics(
        robot_id,
        end_effector_link_index,
        target_position,
        targetOrientation=target_orientation
    )

    # Apply the calculated joint positions
    for i, joint_index in enumerate(movable_joints):
        p.setJointMotorControl2(robot_id, joint_index, p.POSITION_CONTROL, ik_joint_positions[i])

    # Step the simulation
    p.stepSimulation()

    # Get the actual end-effector position and orientation
    link_state = p.getLinkState(robot_id, end_effector_link_index)
    end_effector_position = link_state[4]  # World position
    end_effector_orientation = p.getEulerFromQuaternion(link_state[5])  # Convert quaternion to Euler angles

    # Display the target and end-effector position and orientation
    target_text = "Target Position: x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        target_x, target_y, target_z, target_roll, target_pitch, target_yaw
    )
    end_effector_text = "End Effector (FK): x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        end_effector_position[0], end_effector_position[1], end_effector_position[2],
        end_effector_orientation[0], end_effector_orientation[1], end_effector_orientation[2]
    )
    
    joint_angles=[]
    # Display and print joint angles
    for j, joint_index in enumerate(movable_joints):
        joint_angle = p.getJointState(robot_id, joint_index)[0]  # Get the angle of each joint
        joint_angles.append(joint_angle)
        joint_angle_text = "Joint {} Angle: {:.3f}".format(joint_index, joint_angle)
    

    
        # Update joint angle display text on the screen
        if joint_angle_text_ids[j] is None:
            joint_angle_text_ids[j] = p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0)
        else:
            p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0, replaceItemUniqueId=joint_angle_text_ids[j])    
    
    src_joint_angles = joint_angles[:5]
    a,b,c,d,e = src_joint_angles
    if b<0:
        b = -3.1459+abs(b)
    else:
        b=3.14159-b;
        
    current_time = time.time()
    if current_time - last_command_time >= command_interval:
        sendCommand2(a, b, c, d, e)  # Send the command to the real arm
        last_command_time = current_time  # Update the last command time
        last_command = [a,b,c,d,e]
       
  
    
    # Update the displayed text
    if target_text_id is None:
        target_text_id = p.addUserDebugText(target_text, [0, 0, 1.5], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(target_text, [-0.5, -0.50, 1.5], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=target_text_id)

    if end_effector_text_id is None:
        end_effector_text_id = p.addUserDebugText(end_effector_text, [0, 0, 1.3], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(end_effector_text, [-0.5, -0.5, 1.3], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=end_effector_text_id)

    # Add some delay to slow down the simulation
    time.sleep(0.01)
    
    if mae(end_effector_position,target_position) <= 0.001:
        p.disconnect()
        print(f"Source Target position: {target_position}")
        print(f"Source End Effector position: {end_effector_position}")
        print(f'Source Joint Angles: {joint_angles}')
        src_joint_angles = joint_angles[:5]
        a,b,c,d,e = src_joint_angles
        sendCommand2(a,b,c,d,e)
        time.sleep(3)
        serialInst.write('GRAB\n'.encode());
        time.sleep(5)
        break
        
#############################################################################################

last_command = [0,0,0,0,0]
print(dest)
### Destination
# Connect to PyBullet physics simulation with GUI
p.connect(p.GUI)

# Set the path to PyBullet's built-in data
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the robot URDF model (replace 'urdf_file' with the actual URDF file path)
#urdf = "urdf_file.urdf"  # Replace with the actual URDF file path
robot_id = p.loadURDF(urdf, useFixedBase=True)

# Optionally, load a plane to give the scene a ground
plane_id = p.loadURDF("plane.urdf")

# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

# Set initial camera position
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])

# Get the number of joints in the robot
num_joints = p.getNumJoints(robot_id)

# Filter out the fixed joints
movable_joints = []
for i in range(num_joints):
    joint_info = p.getJointInfo(robot_id, i)
    #print(joint_info)
    joint_type = joint_info[2]
    if joint_type not in [p.JOINT_FIXED]:  # Only add non-fixed joints
        movable_joints.append(i)

# Create sliders for real-time input of end-effector's x, y, z coordinates (IK control)
x_slider = p.addUserDebugParameter("End Effector X", -1, 1, 0)
y_slider = p.addUserDebugParameter("End Effector Y", -1, 1, 0)
z_slider = p.addUserDebugParameter("End Effector Z", 0, 1, 0.5)

# Create sliders for end-effector orientation (roll, pitch, yaw)
roll_slider = p.addUserDebugParameter("End Effector Roll", -3.14, 3.14, 0)  # Roll in radians
pitch_slider = p.addUserDebugParameter("End Effector Pitch", -3.14, 3.14, 0)  # Pitch in radians
yaw_slider = p.addUserDebugParameter("End Effector Yaw", -3.14, 3.14, 0)  # Yaw in radians

# Define end effector link index (adjust based on your URDF's end effector)
end_effector_link_index = 18  # Modify this based on your actual end-effector link index

# Create a visual sphere to represent the target position
target_visual_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[1, 0, 0, 1])  # Red sphere
target_sphere = p.createMultiBody(baseVisualShapeIndex=target_visual_shape)

# Initialize variables for text IDs
target_text_id = None
end_effector_text_id = None
joint_angle_text_ids = [None] * len(movable_joints)  # For joint angle display text IDs

# Main simulation loop
while True:
    # Read the target position from the sliders
#     target_x = p.readUserDebugParameter(x_slider)
#     target_y = p.readUserDebugParameter(y_slider)
#     target_z = p.readUserDebugParameter(z_slider)

    target_x = dest[0]
    target_y = dest[1]+0.045+2*0.028
    target_z = 0.1
    target_position = [target_x, target_y, target_z]

    # Read the target orientation from the sliders
#     target_roll = p.readUserDebugParameter(roll_slider)
#     target_pitch = p.readUserDebugParameter(pitch_slider)
#     target_yaw = p.readUserDebugParameter(yaw_slider)
    target_roll = -0.8
    target_pitch = 0
    target_yaw = 0

    # Convert roll, pitch, yaw to quaternion
    target_orientation = p.getQuaternionFromEuler([target_roll, target_pitch, target_yaw])

    # Move the visual sphere to the target position
    p.resetBasePositionAndOrientation(target_sphere, target_position, [0, 0, 0, 1])

    # Use inverse kinematics to calculate joint positions
    ik_joint_positions = p.calculateInverseKinematics(
        robot_id,
        end_effector_link_index,
        target_position,
        targetOrientation=target_orientation
    )

    # Apply the calculated joint positions
    for i, joint_index in enumerate(movable_joints):
        p.setJointMotorControl2(robot_id, joint_index, p.POSITION_CONTROL, ik_joint_positions[i])

    # Step the simulation
    p.stepSimulation()

    # Get the actual end-effector position and orientation
    link_state = p.getLinkState(robot_id, end_effector_link_index)
    end_effector_position = link_state[4]  # World position
    end_effector_orientation = p.getEulerFromQuaternion(link_state[5])  # Convert quaternion to Euler angles

    # Display the target and end-effector position and orientation
    target_text = "Target Position: x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        target_x, target_y, target_z, target_roll, target_pitch, target_yaw
    )
    end_effector_text = "End Effector (FK): x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        end_effector_position[0], end_effector_position[1], end_effector_position[2],
        end_effector_orientation[0], end_effector_orientation[1], end_effector_orientation[2]
    )
    
    joint_angles=[]
    # Display and print joint angles
    for j, joint_index in enumerate(movable_joints):
        joint_angle = p.getJointState(robot_id, joint_index)[0]  # Get the angle of each joint
        joint_angles.append(joint_angle)
        joint_angle_text = "Joint {} Angle: {:.3f}".format(joint_index, joint_angle)
    

    
        # Update joint angle display text on the screen
        if joint_angle_text_ids[j] is None:
            joint_angle_text_ids[j] = p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0)
        else:
            p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0, replaceItemUniqueId=joint_angle_text_ids[j])    
    
    src_joint_angles = joint_angles[:5]
    a,b,c,d,e = src_joint_angles
    if b<0:
        b = -3.1459+abs(b)
    else:
        b=3.14159-b;
        
    current_time = time.time()
    if current_time - last_command_time >= command_interval:
        sendCommand2(a, b, c, d, e)  # Send the command to the real arm
        last_command_time = current_time  # Update the last command time
        last_command = [a,b,c,d,e]
       
  
    
    # Update the displayed text
    if target_text_id is None:
        target_text_id = p.addUserDebugText(target_text, [0, 0, 1.5], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(target_text, [-0.5, -0.50, 1.5], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=target_text_id)

    if end_effector_text_id is None:
        end_effector_text_id = p.addUserDebugText(end_effector_text, [0, 0, 1.3], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(end_effector_text, [-0.5, -0.5, 1.3], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=end_effector_text_id)

    # Add some delay to slow down the simulation
    time.sleep(1 / 240)
    
    if mae(end_effector_position,target_position) <= 0.005:
        p.disconnect()
        print(f"Source Target position: {target_position}")
        print(f"Source End Effector position: {end_effector_position}")
        print(f'Source Joint Angles: {joint_angles}')
        src_joint_angles = joint_angles[:5]
        a,b,c,d,e = src_joint_angles
        sendCommand2(a,b,c,d,e)
        time.sleep(3)
        serialInst.write('OPEN\n'.encode());
        time.sleep(5)
        break
        
#############################################################################################
sendCommand2(0,0,0,0,0)

[0.056, 0.168]
Source Target position: [0.056, 0.269, 0.1]
Source End Effector position: (0.05536800995469093, 0.26780301332473755, 0.09889485687017441)
Source Joint Angles: [-0.2847638922001396, -0.2183027763169501, 0.13796978913081398, 0.4220116190704958, -0.2028423227724917, -0.00023894877783521527, 0.2011892320400808]
[-0.084, 0.084]
Source Target position: [-0.084, 0.185, 0.1]
Source End Effector position: (-0.07822658121585846, 0.19233594834804535, 0.10164295881986618)
Source Joint Angles: [0.26881449993787304, 0.2068637784182235, 0.57982993015208, 0.4074845479093566, 0.19084398176118522, -0.00011457373688796951, -1.0496033444801709e-10]


b'0.000 0.000 0.000 0.000 0.000\n'

In [ ]:
p.disconnect()


In [175]:
serialInst.close()

In [186]:
sendCommand2(0,0,0,0,0)

b'0.000 0.000 0.000 0.000 0.000\n'

In [192]:
serialInst.write("OPEN\n".encode())

5